In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
chunksize = 10**6

chunk_list = []  # 각 청크를 저장할 리스트

# 청크 단위로 데이터 읽기
for chunk in pd.read_csv('G:/내 드라이브/Colab Notebooks/빅콘(2024)/OD_data.csv', chunksize=chunksize):
    # 각 청크에 대해 원하는 데이터 처리
    chunk_list.append(chunk)

# 병합한 데이터프레임 만들기
od = pd.concat(chunk_list)

In [ ]:
festival = pd.read_csv('G:/내 드라이브/Colab Notebooks/빅콘(2024)/festival_df_찐최종.csv')

In [ ]:
nonseoul_festival = festival[festival['시도명'] != '서울특별시']

In [ ]:
nonseoul_festival

,축제명,축제시작일자,축제종료일자,소재지도로명주소,소재지지번주소,위도,경도,시도명,시군구명,읍면동명,hdong_cd,법정동코드,동리명
0,2023 춘천연극제,2023-07-01,2023-10-28,강원특별자치도 춘천시 서부대성로 71,강원특별자치도 춘천시 옥천동 73-2,37.882374,127.731332,강원특별자치도,춘천시,소양동,5111058000,5111010700,옥천동
1,동강국제사진제,2023-07-21,2023-09-24,강원특별자치도 영월군 영월읍 영월로 1909-10,강원특별자치도 영월군 영월읍 하송리 217-2,37.181940,128.461551,강원특별자치도,영월군,영월읍,5175025000,5175025022,하송리
2,강진하맥축제,2023-08-31,2023-09-02,NaN,전라남도 강진군 군동면 호계리 1-2,34.643545,126.804124,전라남도,강진군,군동면,4681031000,4681031022,호계리
3,괴산고추축제,2023-08-31,2023-09-03,충청북도 괴산군 괴산읍 임꺽정로 92,NaN,36.815410,127.786600,충청북도,괴산군,괴산읍,4376025000,4376025022,서부리
5,제23회 섬강축제,2023-09-01,2023-09-03,강원도 원주시 소금산길 12,강원도 원주시 지정면 간현리 1056-24,37.364576,127.834038,강원특별자치도,원주시,지정면,5113033000,5113033021,간현리
...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,동두천시 자원봉사 축제,2023-10-14,2023-10-14,경기도 동두천시 중앙로 125,경기도 동두천시 지행동 691-5,37.892755,127.051746,경기도,동두천시,송내동,4125056600,4125010200,지행동
102,문경사과축제,2023-10-14,2023-10-29,경상북도 문경시 문경읍 새재로932,경상북도 문경시 문경읍 상초리 288-1,36.761602,128.076990,경상북도,문경시,문경읍,4728025000,4728025030,상초리
106,달구벌목민관축제,2023-10-15,2023-10-15,대구광역시 서구 국채보상로 257,대구광역시 서구 평리동 1230-9,35.872322,128.559266,대구광역시,서구,평리3동,2717064000,2717010300,평리동
107,미소&행복 구민축제,2023-10-15,2023-10-15,대구광역시 서구 문화로 96,대구광역시 서구 이현동 산28-12,35.873615,128.542905,대구광역시,서구,상중이동,2717067500,2717010600,이현동


In [ ]:
od.head()

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
0,1130563500,1130559500,20230901,12:00,13:00,1,3,0.0,1.0,1,10869,58,7
1,2714072000,2714073000,20230901,12:00,12:00,1,4,0.0,0.0,0,2018,4,16
2,3017055500,3017058700,20230901,18:00,18:00,1,2,0.0,4.0,0,14070,39,8
3,2917067300,2917059000,20230901,19:00,19:00,0,4,0.0,0.0,0,2738,9,22
4,2714058000,4729025300,20230901,21:00,22:00,1,1,2.0,3.0,0,43707,85,10


축제 기간별로 전후를 비교할 것
즉, 10일간 축제 진행시, 축제전 10일과 축제 기간 10일간의 방문객을 비교해보겠음

In [ ]:
od['date'] = pd.to_datetime(od['date'], format='%Y%m%d')

In [ ]:
od_pp = od.drop(['origin_hdong_cd','start_time','end_time','origin_purpose','modal'],axis=1)
od_pp.head()

,dest_hdong_cd,date,gender,age,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
0,1130559500,2023-09-01,1,3,1,10869,58,7
1,2714073000,2023-09-01,1,4,0,2018,4,16
2,3017058700,2023-09-01,1,2,0,14070,39,8
3,2917059000,2023-09-01,0,4,0,2738,9,22
4,4729025300,2023-09-01,1,1,0,43707,85,10


In [ ]:
od_pp.isna().sum()

dest_hdong_cd      0
date               0
gender             0
age                0
dest_purpose       0
od_dist_avg        0
od_duration_avg    0
od_cnts            0
dtype: int64

In [ ]:
od1 = od_pp.groupby(['date', 'gender', 'age', 'dest_purpose','dest_hdong_cd'], as_index=False).agg({'od_cnts': 'mean','od_dist_avg':'mean','od_duration_avg':'mean'})

In [ ]:
festival['축제시작일자'] = pd.to_datetime(festival['축제시작일자'])
festival['축제종료일자'] = pd.to_datetime(festival['축제종료일자'])
festival.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   축제명       109 non-null    object        
 1   축제시작일자    109 non-null    datetime64[ns]
 2   축제종료일자    109 non-null    datetime64[ns]
 3   소재지도로명주소  95 non-null     object        
 4   소재지지번주소   85 non-null     object        
 5   위도        109 non-null    float64       
 6   경도        109 non-null    float64       
 7   시도명       109 non-null    object        
 8   시군구명      109 non-null    object        
 9   읍면동명      109 non-null    object        
 10  hdong_cd  109 non-null    int64         
 11  법정동코드     109 non-null    int64         
 12  동리명       109 non-null    object        
dtypes: datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 11.2+ KB


In [ ]:
festival[(festival['축제시작일자'] < '2023-09-01') | (festival['축제종료일자'] > '2023-10-15')]

,축제명,축제시작일자,축제종료일자,소재지도로명주소,소재지지번주소,위도,경도,시도명,시군구명,읍면동명,hdong_cd,법정동코드,동리명,duration
0,2023 춘천연극제,2023-07-01,2023-10-28,강원특별자치도 춘천시 서부대성로 71,강원특별자치도 춘천시 옥천동 73-2,37.882374,127.731332,강원특별자치도,춘천시,소양동,5111058000,5111010700,옥천동,120
1,동강국제사진제,2023-07-21,2023-09-24,강원특별자치도 영월군 영월읍 영월로 1909-10,강원특별자치도 영월군 영월읍 하송리 217-2,37.181940,128.461551,강원특별자치도,영월군,영월읍,5175025000,5175025022,하송리,66
2,강진하맥축제,2023-08-31,2023-09-02,NaN,전라남도 강진군 군동면 호계리 1-2,34.643545,126.804124,전라남도,강진군,군동면,4681031000,4681031022,호계리,3
3,괴산고추축제,2023-08-31,2023-09-03,충청북도 괴산군 괴산읍 임꺽정로 92,NaN,36.815410,127.786600,충청북도,괴산군,괴산읍,4376025000,4376025022,서부리,4
39,아트밸리 아산 제2회 신정호 국제 아트페스티벌 100인 100색전,2023-09-22,2023-12-25,충청남도 아산시 신정로 616,충청남도 아산시 방축동 5,36.771521,126.983915,충청남도,아산시,온양4동,4420060000,4420010300,방축동,95
40,제5회 인제가을꽃축제,2023-09-22,2023-10-22,NaN,강원도 인제군 북면 용대리 1821-1 일원,37.997726,128.098499,강원특별자치도,인제군,북면,5181032000,5181032024,용대리,31
90,강진만 춤추는 갈대축제,2023-10-13,2023-10-22,전라남도 강진군 강진읍 남당로 97-23,전라남도 강진군 강진읍 목리 128-1,34.625971,126.772432,전라남도,강진군,강진읍,4681025000,4681025025,목리,10
92,보은대추축제,2023-10-13,2023-10-22,충청북도 보은군 보은읍 뱃들로 54,NaN,36.487090,127.723700,충청북도,보은군,보은읍,4372025000,4372025031,이평리,10
102,문경사과축제,2023-10-14,2023-10-29,경상북도 문경시 문경읍 새재로932,경상북도 문경시 문경읍 상초리 288-1,36.761602,128.076990,경상북도,문경시,문경읍,4728025000,4728025030,상초리,16


In [ ]:
len(festival[(festival['축제시작일자'] < '2023-09-01') | (festival['축제종료일자'] > '2023-10-15')])

9

In [ ]:
# '축제시작일자'가 '2023-09-01'보다 이른 경우 '2023-09-01'로 수정
festival.loc[festival['축제시작일자'] < '2023-09-01', '축제시작일자'] = '2023-09-01'

# '축제종료일자'가 '2023-10-15'보다 늦은 경우 '2023-10-15'로 수정
festival.loc[festival['축제종료일자'] > '2023-10-15', '축제종료일자'] = '2023-10-15'


In [ ]:
festival['duration'] = (festival['축제종료일자'] - festival['축제시작일자']).dt.days + 1

In [ ]:
festival['duration'].describe()

count    109.000000
mean       3.981651
std        5.894099
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max       45.000000
Name: duration, dtype: float64

전 후를 비교해야하는데, 그게 안되는 경우는 일단 제외함
9월 1일부터 10월 15일 : 45일

In [ ]:
festival_filter = festival[festival['duration'] <= 23] #전후비교가 되려면 이거보단 작아야함.
festival_filter.reset_index(inplace=True)

In [ ]:
festival_filter = festival_filter.drop('index',axis=1)
festival_filter

,축제명,축제시작일자,축제종료일자,소재지도로명주소,소재지지번주소,위도,경도,시도명,시군구명,읍면동명,hdong_cd,법정동코드,동리명,duration
0,강진하맥축제,2023-09-01,2023-09-02,NaN,전라남도 강진군 군동면 호계리 1-2,34.643545,126.804124,전라남도,강진군,군동면,4681031000,4681031022,호계리,2
1,괴산고추축제,2023-09-01,2023-09-03,충청북도 괴산군 괴산읍 임꺽정로 92,NaN,36.815410,127.786600,충청북도,괴산군,괴산읍,4376025000,4376025022,서부리,3
2,2023 군자 한마당축제,2023-09-01,2023-09-01,서울특별시 광진구 능동로 209(군자동),서울특별시 광진구 군자동 98,37.550870,127.074100,서울특별시,광진구,군자동,1121573000,1121510900,군자동,1
3,제23회 섬강축제,2023-09-01,2023-09-03,강원도 원주시 소금산길 12,강원도 원주시 지정면 간현리 1056-24,37.364576,127.834038,강원특별자치도,원주시,지정면,5113033000,5113033021,간현리,3
4,남양열린한마당축제,2023-09-02,2023-09-02,경상남도 사천시 임내안길 65,NaN,34.970370,128.063000,경상남도,사천시,남양동,4824059500,4824012600,송포동,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,제2회 숲나루 꿈의 축제,2023-10-14,2023-10-14,NaN,서울특별시 광진구 구의동 175-1,37.547640,127.097200,서울특별시,광진구,구의제2동,1121586000,1121510300,구의동,1
101,제3회 능골산문화축제,2023-10-14,2023-10-14,서울특별시 양천구 남부순환로64길 2,서울특별시 양천구 신월동 267번지,37.528202,126.832193,서울특별시,양천구,신월3동,1147058000,1147010300,신월동,1
102,달구벌목민관축제,2023-10-15,2023-10-15,대구광역시 서구 국채보상로 257,대구광역시 서구 평리동 1230-9,35.872322,128.559266,대구광역시,서구,평리3동,2717064000,2717010300,평리동,1
103,미소&행복 구민축제,2023-10-15,2023-10-15,대구광역시 서구 문화로 96,대구광역시 서구 이현동 산28-12,35.873615,128.542905,대구광역시,서구,상중이동,2717067500,2717010600,이현동,1


In [ ]:
festival_filter['duration'].describe()

count    105.000000
mean       3.019048
std        2.720934
min        1.000000
25%        1.000000
50%        3.000000
75%        3.000000
max       17.000000
Name: duration, dtype: float64

In [ ]:
chunk_size = 10^5
merged_data = []

# od_pp 데이터를 청크 단위로 나눔
num_chunks = int(np.ceil(len(od1) / chunk_size))

for i in range(num_chunks):
    # od_pp 데이터를 청크로 나눔
    od_pp_chunk = od1[i * chunk_size:(i + 1) * chunk_size]

    # 청크 단위로 festival 데이터와 병합
    df_chunk_merged = pd.merge(od_pp_chunk, festival_filter, how='inner', left_on='dest_hdong_cd', right_on='hdong_cd')

    # 병합한 결과를 리스트에 추가
    merged_data.append(df_chunk_merged)

df_festival = pd.concat(merged_data, ignore_index=True)

In [ ]:
df_festival.to_csv('G:/내 드라이브/Colab Notebooks/빅콘(2024)/festival_merged.csv')

In [ ]:
df_festival['date'] = pd.to_datetime(df_festival['date'])  # date를 datetime 형식으로 변환
df_festival['축제시작일자'] = pd.to_datetime(df_festival['축제시작일자'])  # 축제 시작일도 datetime 형식으로 변환
df_festival['duration'] = pd.to_timedelta(df_festival['duration'], unit='d')  # duration을 일(day) 단위로 변환
df_festival['축제종료일자'] = pd.to_datetime(df_festival['축제종료일자'])


In [ ]:
df_festival['duration']

0        1 days
1        1 days
2        1 days
3        1 days
4        1 days
          ...  
222797   1 days
222798   2 days
222799   4 days
222800   5 days
222801   3 days
Name: duration, Length: 222802, dtype: timedelta64[ns]

In [ ]:
# 축제 전후 데이터를 분리하는 함수
def compare_od_cnts_before_after(df):
    results = []

    # 축제별로 그룹화
    for 축제명, group in df.groupby('축제명'):
        축제_시작 = group['축제시작일자'].min()  # 축제 시작일자
        축제_종료 = group['축제종료일자'].max()  # 축제 종료일자
        축제_전_기간_시작 = 축제_시작 - group['duration'].max()  # 축제 시작일 이전 duration만큼의 기간

        # 축제 전 데이터: 축제 시작일 이전 duration만큼의 데이터
        before_festival = group[(group['date'] >= 축제_전_기간_시작) & (group['date'] < 축제_시작)]

        # 축제 기간 데이터: 축제 시작일 ~ 축제 종료일 데이터
        during_festival = group[(group['date'] >= 축제_시작) & (group['date'] <= 축제_종료)]

        # 축제 전후 od_cnts 합과 평균 계산
        before_sum = before_festival['od_cnts'].sum()
        before_avg = before_festival['od_cnts'].mean()
        during_sum = during_festival['od_cnts'].sum()
        during_avg = during_festival['od_cnts'].mean()

        # 변화율 계산
        cnts_sum_rate = ((during_sum - before_sum) / before_sum) * 100 if before_sum != 0 else None
        cnts_avg_rate = ((during_avg - before_avg) / before_avg) * 100 if before_avg != 0 else None

        # 결과 저장
        results.append({
            '축제명': 축제명,
            '축제전_od_cnts_합': before_sum,
            '축제전_od_cnts_평균': before_avg,
            '축제기간_od_cnts_합': during_sum,
            '축제기간_od_cnts_평균': during_avg,
            'od_cnts_합_변화율': cnts_sum_rate,
            'od_cnts_평균_변화율': cnts_avg_rate
        })

    return pd.DataFrame(results)


In [ ]:
result_df = compare_od_cnts_before_after(df_festival)

In [ ]:
result_df

,축제명,축제전_od_cnts_합,축제전_od_cnts_평균,축제기간_od_cnts_합,축제기간_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
0,2023 과천공연예술축제,1467.969377,10.123927,1586.309890,9.442321,8.061511,-6.732624
1,2023 관악강감찬축제,2357.681189,11.614193,2444.500721,11.809182,3.682412,1.678887
2,2023 광진가족영화제,1154.429226,9.620244,17126.667689,9.753228,1383.561513,1.382336
3,2023 구리 코스모스 한강예술제,1175.433650,8.771893,1148.435645,8.321997,-2.296855,-5.128830
4,2023 군자 한마당축제,0.000000,NaN,558.844860,8.467346,NaN,NaN
...,...,...,...,...,...,...,...
98,평택 원평나루억새축제,479.845043,9.996772,415.245753,9.656878,-13.462532,-3.400035
99,하동북천코스모스메밀꽃축제,1833.268831,7.482730,3163.450385,8.153223,72.557910,8.960536
100,한성백제문화제,2369.768947,10.303343,2399.357747,10.166770,1.248594,-1.325522
101,횡성더덕축제,297.000000,7.425000,497.896976,6.820507,67.642080,-8.141326


In [ ]:
result_df.sort_values(by='od_cnts_평균_변화율', ascending=True)

,축제명,축제전_od_cnts_합,축제전_od_cnts_평균,축제기간_od_cnts_합,축제기간_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
66,장성 탄탄마을 축제,771.669584,8.768973,634.035818,7.548045,-17.835842,-13.923263
27,김삿갓문화제,269.500000,8.166667,296.650000,7.063095,10.074212,-13.513120
63,이병주하동국제문학제,279.013158,9.621143,274.834300,8.328312,-1.497728,-13.437397
75,제29회 서귀포칠십리축제,1546.541015,9.260725,1335.052283,8.042484,-13.674951,-13.154921
54,여르미오페스티벌,614.781751,9.605965,597.475327,8.659063,-2.815052,-9.857439
...,...,...,...,...,...,...,...
37,문경오미자축제,151.000000,5.592593,632.646037,8.549271,318.970885,52.867755
4,2023 군자 한마당축제,0.000000,NaN,558.844860,8.467346,NaN,NaN
19,강진하맥축제,0.000000,NaN,787.822123,8.381086,NaN,NaN
25,괴산고추축제,0.000000,NaN,1220.521695,8.029748,NaN,NaN


In [ ]:
result_df.sort_values(by='od_cnts_평균_변화율', ascending=False)

,축제명,축제전_od_cnts_합,축제전_od_cnts_평균,축제기간_od_cnts_합,축제기간_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
37,문경오미자축제,151.000000,5.592593,632.646037,8.549271,318.970885,52.867755
10,2023 양천가을문화축제: 제10회 양천북페스티벌,460.940767,8.864246,574.528849,10.840167,24.642664,22.290915
9,2023 양천 가을 문화 축제,900.839977,8.919208,1126.935093,10.835914,25.098255,21.489652
80,제4회 양평부추축제,354.576590,7.236257,711.392214,8.782620,100.631467,21.369653
78,제3회 능골산문화축제,525.344137,9.551712,533.630100,11.117294,1.577245,16.390593
...,...,...,...,...,...,...,...
66,장성 탄탄마을 축제,771.669584,8.768973,634.035818,7.548045,-17.835842,-13.923263
4,2023 군자 한마당축제,0.000000,NaN,558.844860,8.467346,NaN,NaN
19,강진하맥축제,0.000000,NaN,787.822123,8.381086,NaN,NaN
25,괴산고추축제,0.000000,NaN,1220.521695,8.029748,NaN,NaN


In [ ]:
result_df.isna().sum()

축제명                0
축제전_od_cnts_합      0
축제전_od_cnts_평균     4
축제기간_od_cnts_합     0
축제기간_od_cnts_평균    0
od_cnts_합_변화율      4
od_cnts_평균_변화율     4
dtype: int64

In [ ]:
result_df[result_df.isna().any(axis=1)]

,축제명,축제전_od_cnts_합,축제전_od_cnts_평균,축제기간_od_cnts_합,축제기간_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
4,2023 군자 한마당축제,0.0,NaN,558.844860,8.467346,NaN,NaN
19,강진하맥축제,0.0,NaN,787.822123,8.381086,NaN,NaN
25,괴산고추축제,0.0,NaN,1220.521695,8.029748,NaN,NaN
74,제23회 섬강축제,0.0,NaN,172.272222,6.625855,NaN,NaN


In [ ]:
null_row = result_df[result_df.isna().any(axis=1)]['축제명']

In [ ]:
festival_filter[festival_filter['축제명'].isin(null_row.tolist())]

,축제명,축제시작일자,축제종료일자,소재지도로명주소,소재지지번주소,위도,경도,시도명,시군구명,읍면동명,hdong_cd,법정동코드,동리명,duration
0,강진하맥축제,2023-09-01,2023-09-02,NaN,전라남도 강진군 군동면 호계리 1-2,34.643545,126.804124,전라남도,강진군,군동면,4681031000,4681031022,호계리,2
1,괴산고추축제,2023-09-01,2023-09-03,충청북도 괴산군 괴산읍 임꺽정로 92,NaN,36.815410,127.786600,충청북도,괴산군,괴산읍,4376025000,4376025022,서부리,3
2,2023 군자 한마당축제,2023-09-01,2023-09-01,서울특별시 광진구 능동로 209(군자동),서울특별시 광진구 군자동 98,37.550870,127.074100,서울특별시,광진구,군자동,1121573000,1121510900,군자동,1
3,제23회 섬강축제,2023-09-01,2023-09-03,강원도 원주시 소금산길 12,강원도 원주시 지정면 간현리 1056-24,37.364576,127.834038,강원특별자치도,원주시,지정면,5113033000,5113033021,간현리,3


전이 없는 값들은 후로 비교

In [ ]:
null_row

4     2023 군자 한마당축제
19           강진하맥축제
25           괴산고추축제
74        제23회 섬강축제
Name: 축제명, dtype: object

In [ ]:
def compare_od_cnts_after(df, null_row):
    results = []

    # null_row에 있는 축제만 필터링
    df_null = df[df['축제명'].isin(null_row)]

    # 축제별로 그룹화
    for 축제명, group in df_null.groupby('축제명'):
        축제_시작 = group['축제시작일자'].min()  # 축제 시작일자
        축제_종료 = group['축제종료일자'].max()  # 축제 종료일자
        축제_후_기간_끝 = 축제_종료 + group['duration'].max()  # 축제 종료 후 duration만큼의 기간

        # 축제 기간 데이터: 축제 시작일 ~ 축제 종료일 데이터
        during_festival = group[(group['date'] >= 축제_시작) & (group['date'] <= 축제_종료)]

        # 축제 후 데이터: 축제 종료 후 duration만큼의 데이터
        after_festival = group[(group['date'] > 축제_종료) & (group['date'] <= 축제_후_기간_끝)]

        # 축제 전후 od_cnts 합과 평균 계산
        during_sum = during_festival['od_cnts'].sum()
        during_avg = during_festival['od_cnts'].mean()
        after_sum = after_festival['od_cnts'].sum()
        after_avg = after_festival['od_cnts'].mean()

        # 변화율 계산
        cnts_sum_rate = ((during_sum - after_sum) / during_sum) * 100 if during_sum != 0 else None
        cnts_avg_rate = ((during_avg - after_avg) / during_avg) * 100 if during_avg != 0 else None

        # 결과 저장
        results.append({
            '축제명': 축제명,
            '축제기간_od_cnts_합': during_sum,
            '축제기간_od_cnts_평균': during_avg,
            '축제후_od_cnts_합': after_sum,
            '축제후_od_cnts_평균': after_avg,
            'od_cnts_합_변화율': cnts_sum_rate,
            'od_cnts_평균_변화율': cnts_avg_rate
        })

    return pd.DataFrame(results)


In [ ]:
after_df = compare_od_cnts_after(df_festival,null_row)

In [ ]:
after_df

,축제명,축제기간_od_cnts_합,축제기간_od_cnts_평균,축제후_od_cnts_합,축제후_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
0,2023 군자 한마당축제,558.844860,8.467346,511.629993,8.527167,8.448654,-0.706481
1,강진하맥축제,787.822123,8.381086,425.119048,7.458229,46.038701,11.011192
2,괴산고추축제,1220.521695,8.029748,1055.612550,8.058111,13.511365,-0.353225
3,제23회 섬강축제,172.272222,6.625855,400.594913,7.558395,-132.535988,-14.074258


In [ ]:
result_df.dropna(inplace=True)
result_df.to_csv('G:\내 드라이브\Colab Notebooks\빅콘(2024)\comparing.csv')
after_df.to_csv('G:\내 드라이브\Colab Notebooks\빅콘(2024)\comparing_after.csv')

In [ ]:
result_df[result_df['od_cnts_평균_변화율']>0].sort_values(by='od_cnts_평균_변화율', ascending=True)

,축제명,축제전_od_cnts_합,축제전_od_cnts_평균,축제기간_od_cnts_합,축제기간_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
7,2023 안양춤축제 ANYANG DANCE FESTIVAL,2431.356210,9.964575,2381.853217,9.965913,-2.036024,0.013432
17,강릉커피축제,2201.733946,7.514450,2263.231777,7.519042,2.793155,0.061111
20,경북영주 풍기인삼축제,2337.141385,7.490838,2962.276520,7.499434,26.747853,0.114760
30,달구벌목민관축제,391.508726,7.386957,415.012445,7.410937,6.003370,0.324618
97,평창효석문화제,1908.775994,9.089409,3164.449568,9.119451,65.784229,0.330513
11,2023 원주 댄싱카니발,700.926188,6.550712,861.393526,6.575523,22.893614,0.378754
58,용마폭포 문화예술축제,1010.731492,8.565521,946.236480,8.602150,-6.381023,0.427630
86,진주 전국민속 소힘겨루기대회,3220.442774,10.879874,3247.449841,10.934175,0.838613,0.499090
83,제7회 구삼축제 한마당,482.390324,9.458634,523.582322,9.519679,8.539143,0.645387
26,구민화합대잔치,407.138860,7.983115,417.962534,8.037741,2.658472,0.684271


In [ ]:
result_df[result_df['od_cnts_평균_변화율']<0].sort_values(by='od_cnts_평균_변화율', ascending=True)

,축제명,축제전_od_cnts_합,축제전_od_cnts_평균,축제기간_od_cnts_합,축제기간_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
66,장성 탄탄마을 축제,771.669584,8.768973,634.035818,7.548045,-17.835842,-13.923263
27,김삿갓문화제,269.500000,8.166667,296.650000,7.063095,10.074212,-13.513120
63,이병주하동국제문학제,279.013158,9.621143,274.834300,8.328312,-1.497728,-13.437397
75,제29회 서귀포칠십리축제,1546.541015,9.260725,1335.052283,8.042484,-13.674951,-13.154921
54,여르미오페스티벌,614.781751,9.605965,597.475327,8.659063,-2.815052,-9.857439
40,사천 문화재 야행,1467.072653,9.226872,1249.951247,8.333008,-14.799636,-9.687614
41,사천읍성 문화축제,901.387826,9.013878,876.326716,8.189969,-2.780281,-9.140449
95,태백 황지연못 황부자며느리 축제,1142.530381,8.463188,854.427573,7.697546,-25.216205,-9.046736
101,횡성더덕축제,297.000000,7.425000,497.896976,6.820507,67.642080,-8.141326
62,의왕백운호수축제,559.451215,7.459350,535.439495,6.864609,-4.292013,-7.973090


In [ ]:
len(result_df[result_df['od_cnts_평균_변화율']<0])

54

In [ ]:
after_df[after_df['od_cnts_평균_변화율']>0]

,축제명,축제기간_od_cnts_합,축제기간_od_cnts_평균,축제후_od_cnts_합,축제후_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
0,2023 군자 한마당축제,558.844860,8.467346,511.629993,8.527167,-8.448654,0.706481
2,괴산고추축제,1220.521695,8.029748,1055.612550,8.058111,-13.511365,0.353225
3,제23회 섬강축제,172.272222,6.625855,400.594913,7.558395,132.535988,14.074258


In [ ]:
after_df[after_df['od_cnts_평균_변화율']<0]

,축제명,축제기간_od_cnts_합,축제기간_od_cnts_평균,축제후_od_cnts_합,축제후_od_cnts_평균,od_cnts_합_변화율,od_cnts_평균_변화율
1,강진하맥축제,787.822123,8.381086,638.230159,7.421281,-18.988038,-11.452042


duration 중에 9월 1일로 cut, 10월 15일로 cut ->변경 완료

1. 나한테는 의미O . 오빠 X -> filter & 반대도
2. 나한테 의미 O -> 지역 filter